In [2]:
import os
import pandas as pd
import gradio as gr
import re
from sentence_transformers import CrossEncoder
from collections import Counter
from bs4 import BeautifulSoup
import plotly.express as px

# Модель

In [ ]:
import google.generativeai as genai

GOOGLE_API_KEY = os.getenv('GOOGLE_API')
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY не установлен. Пожалуйста, установите его как переменную окружения.")

genai.configure(api_key=GOOGLE_API_KEY)

# === 1. Выбор модели Gemini ===
# Gemini 1.5 Flash - это "gemini-1.5-flash-latest" или "gemini-1.5-flash"
GEMINI_MODEL_NAME = "gemini-1.5-flash-latest"

# Инициализация модели Gemini
# safety_settings по умолчанию довольно строгие, можно ослабить, если нужно
generation_config = {
    "temperature": 0.3, # Контроль "креативности", 0.3 - довольно конкретный
    "top_p": 0.95,      # Контроль разнообразия
    "max_output_tokens": 2000, # Максимальное количество токенов в ответе Gemini
}

# Инициализация модели, передавая конфигурацию генерации
# Это позволит не передавать параметры каждый раз при вызове generate_content
gemini_model = genai.GenerativeModel(
    model_name=GEMINI_MODEL_NAME,
    generation_config=generation_config
)


In [4]:
def ask_gemini_model(question, context=""):
    context = "\n---\n".join([desc for desc in df['description']])
    messages = [
        {"role": "user", "parts": [
            "Ты — полезный AI-помощник, который анализирует информацию из описаний вакансий.\n"
            "Отвечай на вопросы, основываясь СТРОГО на предоставленном КОНТЕКСТЕ.\n"
        ]},
        {"role": "model", "parts": ["Понял. Я готов отвечать на вопросы строго по предоставленному контексту о вакансиях."]},
        {"role": "user", "parts": [
            f"КОНТЕКСТ:\n{context}\n\nВОПРОС: {question}"
        ]}
    ]

    try:
        # Вызов API Gemini
        response = gemini_model.generate_content(messages)
        return response.text.strip()
    except Exception as e:
        # Если модель не смогла сгенерировать ответ (например, из-за safety settings или ошибки API)
        print(f"Ошибка при запросе к Gemini API: {e}")
        # Если response.prompt_feedback содержит block_reason, это из-за Safety Settings
        if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
            return f"Ответ заблокирован из-за настроек безопасности: {response.prompt_feedback.block_reason}"
        return f"Неизвестная ошибка: {e}"

# Отображение

## Зарплаты

In [5]:
def clean_salary_value(salary_str):
    cleaned = re.sub(r'[^\d]', '', salary_str)

    try:
        if cleaned: # Проверяем, что строка не пуста после очистки
            return int(cleaned)
        else:
            return None
    except ValueError:
        return None

def plot_salaries():
    global df # Убедимся, что работаем с глобальным df

    if 'df' not in globals() or df.empty:
        print("DEBUG: DataFrame пуст или не инициализирован в plot_salaries.")
        fig = px.bar(title="График зарплат (Загрузите вакансии)")
        fig.update_layout(xaxis_title="Вакансия", yaxis_title="Зарплата")
        return fig


    salaries_processed = []
    for salary_entry in df['salary']:
        from_val = None
        to_val = None
        if isinstance(salary_entry, str):
            if 'от' in salary_entry:
                parts = salary_entry.split('от')
                if len(parts) > 1:
                    from_val = clean_salary_value(parts[1])
            elif 'до' in salary_entry:
                parts = salary_entry.split('до')
                if len(parts) > 1:
                    to_val = clean_salary_value(parts[1])
            else:
                # Попробуем извлечь первое число, если нет "от" или "до"
                numbers = re.findall(r'\d+', salary_entry)
                if numbers:
                    from_val = clean_salary_value(numbers[0])
                    if len(numbers) > 1: # Если есть диапазон, возьмем второе число как "до"
                        to_val = clean_salary_value(numbers[1])

        salaries_processed.append({'salary_from_val': from_val, 'salary_to_val': to_val})

    salary_df_processed = pd.DataFrame(salaries_processed)

    plot_df = pd.DataFrame()
    plot_df['Вакансия'] = df['name']
    plot_df['Зарплата (начало)'] = salary_df_processed['salary_from_val']
    plot_df['Зарплата (конец)'] = salary_df_processed['salary_to_val']

    # Если 'salary_from_val' пуст, но есть 'salary_to_val', используем 'salary_to_val' как 'начало'
    plot_df['Зарплата (начало)'] = plot_df['Зарплата (начало)'].fillna(plot_df['Зарплата (конец)'])
    # Если 'salary_to_val' пуст, но есть 'salary_from_val', используем 'salary_from_val' как 'конец'
    plot_df['Зарплата (конец)'] = plot_df['Зарплата (конец)'].fillna(plot_df['Зарплата (начало)'])

    plot_df = plot_df[plot_df['Зарплата (начало)'] < 5000000]

    if plot_df.empty:
        print("DEBUG: plot_df пуст после обработки зарплат.")
        fig = px.bar(title="График зарплат (Нет данных о зарплате)",
                     labels={'Вакансия': 'Вакансия', 'Зарплата': 'Зарплата'})
        return fig

    # Создаем график
    fig = px.box(plot_df,
                 y='Зарплата (начало)', # Используем "начало" для высоты столбца
                 hover_data={'Зарплата (конец)': True, 'Вакансия': False}, # Показываем "до" при наведении
                 title='Зарплаты по вакансиям',
                 height=300,
                 labels={'Зарплата (начало)': 'Зарплата'})
    fig.update_layout(xaxis_title="Box plot", yaxis_title="Зарплата")
    return fig

## Уровни вакансий

In [6]:
def analyze_levels():
    global df # Убедимся, что мы работаем с глобальным df

    if 'df' not in globals() or df.empty:
        fig = px.pie(title="Распределение уровней (Загрузите вакансии)")
        fig.update_layout(
            annotations=[
                dict(
                    text="Данные об уровнях отсутствуют или не загружены.",
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14, color="gray")
                )
            ]
        )
        return fig

    # Убедимся, что столбец 'name' существует
    if 'name' not in df.columns:
        fig = px.pie(title="Распределение уровней (Отсутствует столбец 'name')")
        fig.update_layout(
            annotations=[
                dict(
                    text="Для анализа уровней требуется столбец 'name'.",
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14, color="gray")
                )
            ]
        )
        return fig

    levels_data = []
    # Паттерн для поиска уровней в названии вакансии (регистронезависимый)
    # Добавлены границы слов \b для более точного соответствия
    level_pattern = re.compile(r'\b(junior|middle|senior|lead|staff)\b', re.IGNORECASE)

    for index, row in df.iterrows():
        vacancy_name = row['name']
        level = 'Неизвестно' # Значение по умолчанию

        if isinstance(vacancy_name, str):
            match = level_pattern.search(vacancy_name)
            if match:
                detected_level = match.group(0).capitalize()
                # Убедимся, что это одно из ожидаемых слов-уровней
                if detected_level in ['Junior', 'Middle', 'Senior', 'Lead', 'Staff']:
                    level = detected_level
        levels_data.append({'Вакансия': vacancy_name, 'Уровень': level})

    if not levels_data:
        fig = px.pie(title="Распределение уровней (Не удалось определить уровни)")
        fig.update_layout(
            annotations=[
                dict(
                    text="Не удалось определить уровни для загруженных вакансий.",
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14, color="gray")
                )
            ]
        )
        return fig

    levels_df = pd.DataFrame(levels_data)

    # Подсчет количества вакансий для каждого уровня
    level_counts = levels_df['Уровень'].value_counts().reset_index()
    level_counts.columns = ['Уровень', 'Количество']


    # Определяем желаемый порядок отображения уровней на графике
    level_order = ['Junior', 'Middle', 'Senior', 'Lead', 'Staff', 'Неизвестно']

    # Преобразуем столбец 'Уровень' в категориальный тип с заданным порядком
    # Esto asegura que las categorías ausentes no causen errores
    level_counts['Уровень'] = pd.Categorical(level_counts['Уровень'], categories=level_order, ordered=True)

    # Сортируем DataFrame по новому категориальному столбцу для правильного отображения
    level_counts = level_counts.sort_values(by='Уровень')


    if level_counts.empty:
        fig = px.pie(title="Распределение уровней (Нет данных для построения)")
        fig.update_layout(
            annotations=[
                dict(
                    text="Недостаточно данных для построения круговой диаграммы уровней.",
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14, color="gray")
                )
            ]
        )
        return fig


    # Создаем круговую диаграмму с помощью Plotly Express
    fig = px.pie(level_counts,
                 values='Количество',
                 names='Уровень',
                 title='Распределение уровней вакансий по названию',
                 labels={'Уровень': 'Уровень кандидата', 'Количество': 'Количество вакансий'},
                 color='Уровень', # Раскрашиваем секторы по уровням
                 height=300)

    return fig

## Ключевые навыки

In [7]:
def show_skills():
    if 'df' not in globals() or df.empty:
        # Return a placeholder plot if no data
        fig = px.bar(title="Гистограмма навыков (Загрузите вакансии)",
                     labels={'x': 'Навык', 'y': 'Частота'})
        return fig

    all_skills = []
    for skills_list in df['key_skills']:
        if isinstance(skills_list, list):
            for skill_dict in skills_list:
                if isinstance(skill_dict, dict) and 'name' in skill_dict and len(skill_dict['name']) < 40:
                    all_skills.append(skill_dict['name'])

    if not all_skills:
        fig = px.bar(title="Гистограмма навыков (Навыки не найдены)",
                     labels={'x': 'Навык', 'y': 'Частота'})
        return fig

    skill_counts = Counter(all_skills)
    skill_df = pd.DataFrame(skill_counts.items(), columns=['Навык', 'Частота'])
    skill_df = skill_df.sort_values(by='Частота', ascending=False).head(20) # Top 20 skills

    fig = px.bar(skill_df,
                 x='Навык',
                 y='Частота',
                 title='Топ 20 наиболее частых ключевых навыков',
                 labels={'Навык': 'Навык', 'Частота': 'Количество вакансий'},
                 height=300)
    fig.update_layout(xaxis={'categoryorder':'total descending'}) # Order bars by frequency

    return fig

# Пасинг вакансий

In [8]:
HH_API = "https://api.hh.ru/vacancies"

def fetch_vacancies(query, count):
    global df
    # Инициализация всех возвращаемых значений на случай ошибки
    status_message = "Инициализация..."
    skills_output_val = "Навыки не загружены."
    salary_plot_output_val = plot_salaries() # Вызов для получения пустого графика-заглушки
    levels_output_val = "Уровни не загружены."

    try:
        params = {
            "text": query,
            "order_by": "relevance",
            "label": "with_salary",
            "per_page": count,
            "search_field": ["name", "description"],
        }
        response = requests.get(HH_API, params=params)
        response.raise_for_status() # Вызывает HTTPError для статусов 4xx/5xx

        items = response.json().get("items", [])
        vacancies = []

        for item in items:
            try:
                vacancy_url = item['url']
                details = requests.get(vacancy_url).json()

                description = details.get('description', '')
                if description:
                    soup = BeautifulSoup(description, 'html.parser')
                    description = soup.get_text(separator=' ').strip()

                salary = item.get("salary")

                key_skills = details['key_skills']

                if salary:
                    salary_str = f"{salary.get('from', '')} - {salary.get('to', '')} {salary.get('currency', '')}"

                vacancies.append({
                    "name": item.get("name"),
                    "employer": item.get("employer", {}).get("name"),
                    "description": description,
                    "key_skills": key_skills,
                    "salary": salary_str,
                    "url": item.get("alternate_url")
                })
            except Exception as e:
                print(f"Ошибка при обработке отдельной вакансии '{item.get('name', 'N/A')}': {e}")
                continue # Продолжаем обработку других вакансий

        df = pd.DataFrame(vacancies)

        if not df.empty:
            status_message = f'Загружено {len(df)} вакансий успешно!'
            # Если данные загружены, вычисляем реальные значения
            skills_output_val = show_skills()
            salary_plot_output_val = plot_salaries()
            levels_output_val = analyze_levels()
            reqs_output = ask_gemini_model('Составть требования, чтобы получить работу из контекста. Делай общую суммаризацию со всех вакансий')
            strat_output = ask_gemini_model('Составть стратегию, чтобы получить работу из контекста')
            resume_output = ask_gemini_model('Составть шаблон резюме, чтобы получить работу из контекста. Придумай идеального кандидата и напиши для него этот шаблон. Сделай его 1 универсальный насколько можешь')
        else:
            status_message = 'Вакансий не найдено по вашему запросу.'
            # Если вакансий нет, можно оставить заглушки или установить более информативные сообщения
            skills_output_val = "Вакансий не найдено для анализа навыков."
            salary_plot_output_val = plot_salaries() # Вернет пустой график с сообщением
            levels_output_val = "Вакансий не найдено для анализа уровней."

    except requests.exceptions.RequestException as e:
        status_message = f"Ошибка подключения к HH.ru или HTTP-ошибка: {e}"
        print(f"Ошибка HTTP запроса: {e}")
        df = pd.DataFrame() # Обнуляем DataFrame на случай ошибки
    except ValueError as e: # Для ошибок парсинга JSON
        status_message = f"Ошибка парсинга ответа от HH.ru: {e}"
        print(f"Ошибка JSON парсинга: {e}")
        df = pd.DataFrame()
    except Exception as e:
        status_message = f"Произошла непредвиденная ошибка: {e}"
        print(f"Непредвиденная ошибка в fetch_vacancies: {e}")
        df = pd.DataFrame()

    # Гарантируем, что всегда возвращаются 4 значения
    return skills_output_val, salary_plot_output_val, levels_output_val, reqs_output, strat_output, resume_output

In [9]:
def ask_gemini_model(question, context=""):
    context = "\n---\n".join([desc for desc in df['description']])
    messages = [
        {"role": "user", "parts": [
            "Ты — полезный AI-помощник, который анализирует информацию из описаний вакансий.\n"
            "Отвечай на вопросы, основываясь СТРОГО на предоставленном КОНТЕКСТЕ.\n"
        ]},
        {"role": "model", "parts": ["Понял. Я готов отвечать на вопросы строго по предоставленному контексту о вакансиях."]},
        {"role": "user", "parts": [
            f"КОНТЕКСТ:\n{context}\n\nВОПРОС: {question}"
        ]}
    ]

    try:
        # Вызов API Gemini
        response = gemini_model.generate_content(messages)
        return response.text.strip()
    except Exception as e:
        # Если модель не смогла сгенерировать ответ (например, из-за safety settings или ошибки API)
        print(f"Ошибка при запросе к Gemini API: {e}")
        # Если response.prompt_feedback содержит block_reason, это из-за Safety Settings
        if hasattr(response, 'prompt_feedback') and response.prompt_feedback.block_reason:
            return f"Ответ заблокирован из-за настроек безопасности: {response.prompt_feedback.block_reason}"
        return f"Неизвестная ошибка: {e}"

# Интерфейс

In [10]:
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Анализатор вакансий HH.ru")

    with gr.Row():
        query_input = gr.Textbox(label="Поисковый запрос (например, 'ML разработчик')", value="Python разработчик")
        count_input = gr.Number(label="Сколько вакансий загрузить", value=20, step=1, minimum=1, maximum=100)
        load_btn = gr.Button("🔍 Загрузить вакансии")

    # Здесь объявляем компоненты, которые будут отображаться в колонках
    # и привязываем их к кнопке загрузки
    # Важно: объявляем их внутри Row/Column для правильного размещения

    with gr.Row():
        with gr.Column(): # Первая колонка для навыков
            gr.Markdown("## 📊 Анализ ключевых навыков")
            skills_output = gr.Plot(label="Навыки после загрузки")
        with gr.Column(): # Вторая колонка для уровней (можно настроить scale по желанию)
            gr.Markdown("## 📈 Уровни вакансий")
            levels_output = gr.Plot(label="Уровни вакансий")
        with gr.Column():
            gr.Markdown("## 💰 Анализ зарплат")
            salary_plot_output = gr.Plot(label="График зарплат")

    with gr.Row():
      with gr.Column(): # Вторая колонка для уровней (можно настроить scale по желанию)
            gr.Markdown("## Требования работодателей")
            reqs_output = gr.Textbox(label="Требования")
      with gr.Column():
            gr.Markdown("## Стратегия, как найти эту работу")
            strat_output = gr.Textbox(label="Стратегия")

    with gr.Row():
      with gr.Column(): # Вторая колонка для уровней (можно настроить scale по желанию)
            gr.Markdown("## Шаблон резюме")
            resume_output = gr.Textbox(label="Резюме")

    # Привязываем кнопку загрузки ко всем соответствующим выходам
    # Здесь используются переменные, которые были объявлены в своих колонках
    load_btn.click(
        fn=fetch_vacancies,
        inputs=[query_input, count_input],
        outputs=[skills_output, salary_plot_output, levels_output, reqs_output, strat_output, resume_output]
    )

    gr.Markdown("## 💬 Вопросы к загруженным вакансиям")
    question_input = gr.Textbox(label="Задайте вопрос по всем загруженным вакансиям (например, 'Каковы общие требования к опыту работы?' или 'Суммируй все описания вакансий.')")
    question_btn = gr.Button("Получить ответ / Суммаризировать")
    answer_output = gr.Textbox(label="Ответ Gemini")

    question_btn.click(fn=ask_gemini_model, inputs=question_input, outputs=answer_output)

print("Готово! Запускаем интерфейс...")
demo.launch(share=True, debug=True)

Готово! Запускаем интерфейс...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ec0315afb0755942ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ec0315afb0755942ae.gradio.live
